In [1]:
!pip install flask pyngrok nltk
from flask import Flask, request, render_template_string, session
from pyngrok import ngrok
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
app = Flask(__name__)
app.secret_key = "study-pal-secret"

In [3]:
quiz_data = {
    "Math": {
        "Easy": [
            {"question": "What is the perimeter of a square with side 5 cm?",
             "options": ["10 cm", "15 cm", "20 cm", "25 cm"], "answer": "20 cm"},
            {"question": "If x = 2, what is the value of 3x + 4?",
             "options": ["10", "12", "14", "8"], "answer": "10"},
            {"question": "Convert 500 cm to meters.",
             "options": ["5 m", "50 m", "0.5 m", "0.05 m"], "answer": "5 m"},
        ],
        "Moderate": [
            {"question": "12 × 8 = ?", "options": ["96", "98", "108"], "answer": "96"},
            {"question": "Square root of 81?", "options": ["9", "8", "7"], "answer": "9"},
        ],
        "Hard": [
            {"question": "Solve 2x + 5 = 15", "options": ["x = 5", "x = 10", "x = 8"], "answer": "x = 5"},
            {"question": "Approximate value of π?", "options": ["3.12", "3.14", "3.18"], "answer": "3.14"},
        ],
    },
    "Science": {
        "Easy": [
            {"question": "Water freezes at what temperature (°C)?", "options": ["0", "10", "100"], "answer": "0"},
            {"question": "The Sun is a ___", "options": ["Star", "Planet", "Asteroid"], "answer": "Star"},
        ],
        "Moderate": [
            {"question": "Who discovered gravity?", "options": ["Newton", "Einstein", "Tesla"], "answer": "Newton"},
            {"question": "Which planet has rings?", "options": ["Earth", "Mars", "Saturn"], "answer": "Saturn"},
        ],
        "Hard": [
            {"question": "H₂O₂ is known as?", "options": ["Hydrogen Peroxide", "Hydroxide", "Water"],
             "answer": "Hydrogen Peroxide"},
            {"question": "Photosynthesis occurs in which cell organelle?",
             "options": ["Chloroplast", "Nucleus", "Ribosome"], "answer": "Chloroplast"},
        ],
    },
    "English": {
        "Easy": [
            {"question": "Choose the correct synonym for 'Happy'.",
             "options": ["Sad", "Joyful", "Angry", "Tired"], "answer": "Joyful"},
            {"question": "Which is a verb?", "options": ["Run", "Blue", "Table", "Quick"], "answer": "Run"},
        ],
        "Moderate": [
            {"question": "Identify the adjective: 'She has a beautiful dress.'",
             "options": ["She", "Beautiful", "Dress", "Has"], "answer": "Beautiful"},
            {"question": "Plural of 'Child' is?", "options": ["Childs", "Childes", "Children", "Child"], "answer": "Children"},
        ],
        "Hard": [
            {"question": "Select the correct spelling.", "options": ["Occurence", "Occurrance", "Occurrence"], "answer": "Occurrence"},
            {"question": "Choose the correct article: 'He is ___ honest man.'",
             "options": ["a", "an", "the"], "answer": "an"},
        ],
    },
}

In [4]:
index_html = """
<!DOCTYPE html>
<html>
<head>
    <title>AI Study Pal</title>
    <style>
        body {
            font-family: 'Poppins', sans-serif;
            background: linear-gradient(135deg, #a18cd1, #fbc2eb);
            color: #333;
            text-align: center;
            padding: 0;
            margin: 0;
        }

        .container {
            margin-top: 100px;
        }

        h1 {
            font-size: 3rem;
            color: #fff;
            text-shadow: 2px 2px 6px rgba(0,0,0,0.3);
            margin-bottom: 20px;
        }

        p {
            color: #f0f0f0;
            font-size: 1.2rem;
            margin-bottom: 40px;
        }

        .menu-card {
            background: #ffffffaa;
            border-radius: 20px;
            width: 400px;
            margin: auto;
            padding: 30px;
            box-shadow: 0 4px 20px rgba(0,0,0,0.2);
        }

        select, input[type="number"] {
            width: 90%;
            padding: 10px;
            margin: 10px 0;
            border-radius: 10px;
            border: none;
            font-size: 1rem;
            outline: none;
        }

        button {
            background: linear-gradient(90deg, #667eea, #764ba2);
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 10px;
            font-size: 1rem;
            margin-top: 15px;
            cursor: pointer;
            transition: transform 0.2s;
        }

        button:hover {
            transform: scale(1.05);
        }

        .section {
            margin-top: 30px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>📚 AI Study Pal</h1>
        <p>Your Personal Study Assistant for Class 10</p>

        <div class="menu-card">
            <div class="section">
                <h2>🗓️ Create Study Plan</h2>
                <form action="/study_plan" method="post">
                    <label>Subject:</label><br>
                    <select name="subject">
                        <option>Math</option>
                        <option>Science</option>
                        <option>English</option>
                        <option>Social Studies</option>
                        <option>Computer Science</option>
                    </select><br>
                    <label>Hours to Study:</label><br>
                    <input type="number" name="hours" min="1" max="8" required><br>
                    <button type="submit">Generate Plan</button>
                </form>
            </div>

            <div class="section">
                <h2>🧩 Take a Quiz</h2>
                <form action="/quiz" method="post">
                    <label>Subject:</label><br>
                    <select name="subject">
                        <option>Math</option>
                        <option>Science</option>
                        <option>English</option>
                        <option>Social Studies</option>
                        <option>Computer Science</option>
                    </select><br>
                    <label>Difficulty:</label><br>
                    <select name="difficulty">
                        <option>Easy</option>
                        <option>Moderate</option>
                        <option>Hard</option>
                    </select><br>
                    <button type="submit">Start Quiz</button>
                </form>
            </div>
        </div>
    </div>
</body>
</html>
"""


In [5]:
quiz_html = """
<!DOCTYPE html>
<html>
<head>
    <title>{{subject}} Quiz</title>
    <style>
        body {font-family: 'Poppins', sans-serif; background: linear-gradient(135deg,#89f7fe,#66a6ff);
              color:#333; text-align:center; margin:0; padding:40px;}
        form {background:white; border-radius:20px; padding:30px; width:60%; margin:auto;
              box-shadow:0 4px 20px rgba(0,0,0,0.2);}
        h2 {color:#444;}
        .question {margin:20px 0; font-size:1.1rem;}
        button {background:linear-gradient(90deg,#667eea,#764ba2); color:white;
                border:none; padding:12px 25px; border-radius:10px; font-size:1rem; cursor:pointer;}
        button:hover {transform:scale(1.05);}
    </style>
</head>
<body>
    <h1>{{subject}} Quiz ({{difficulty}})</h1>
    <form action="/result" method="post">
        {% for i in range(questions|length) %}
            <div class="question">
                <b>Q{{i+1}}: {{questions[i]["question"]}}</b><br>
                {% for opt in questions[i]["options"] %}
                    <input type="radio" name="q{{i}}" value="{{opt}}" required> {{opt}}<br>
                {% endfor %}
            </div>
        {% endfor %}
        <button type="submit">Submit Quiz</button>
    </form>
</body>
</html>
"""

In [6]:
result_html = """
<!DOCTYPE html>
<html>
<head>
    <title>Quiz Result</title>
    <style>
        body {font-family: 'Poppins', sans-serif; background: linear-gradient(135deg, #f6d365, #fda085);
              color:#333; text-align:center; margin:0; padding:40px;}
        .card {background:white; border-radius:20px; padding:30px; width:60%; margin:auto;
               box-shadow:0 4px 20px rgba(0,0,0,0.2);}
        h2 {color:#444;}
        .correct {color:green; font-weight:bold;}
        .wrong {color:red; font-weight:bold;}
        button {background:linear-gradient(90deg,#667eea,#764ba2); color:white;
                border:none; padding:12px 25px; border-radius:10px; font-size:1rem; cursor:pointer;}
        button:hover {transform:scale(1.05);}
    </style>
</head>
<body>
    <h1>🎯 Quiz Results</h1>
    <div class="card">
        <h2>You scored {{score}} out of {{total}}</h2>
        <hr>
        {% for q in feedback %}
            <div style="margin-bottom:15px;">
                <b>Q{{loop.index}}: {{q["question"]}}</b><br>
                Your answer: <span class="{{ 'correct' if q['is_correct'] else 'wrong' }}">{{q["your_answer"]}}</span><br>
                Correct answer: <b>{{q["correct_answer"]}}</b>
            </div>
        {% endfor %}
        <form action="/" method="get">
            <button type="submit">🏠 Back to Menu</button>
        </form>
    </div>
</body>
</html>
"""

In [7]:
@app.route("/")
def home():
    session.clear()
    return render_template_string(index_html)

@app.route("/study_plan", methods=["POST"])
def study_plan():
    subject = request.form["subject"]
    hours = int(request.form["hours"])
    plan = {f"Hour {i}": f"Study {subject} – Focus on key concepts." for i in range(1, hours+1)}
    return f"<h1>📚 Study Plan for {subject}</h1>" + "<br>".join([f"{k}: {v}" for k,v in plan.items()]) + "<br><a href='/'>Back</a>"

@app.route("/quiz", methods=["POST"])
def quiz():
    subject = request.form["subject"]
    difficulty = request.form["difficulty"]
    session["subject"], session["difficulty"] = subject, difficulty
    session["questions"] = quiz_data[subject][difficulty]
    return render_template_string(quiz_html, subject=subject, difficulty=difficulty, questions=quiz_data[subject][difficulty])
@app.route("/result", methods=["POST"])
def result():
    subject = session.get("subject")
    difficulty = session.get("difficulty")
    questions = quiz_data[subject][difficulty]

    score = 0
    feedback = []

    for i, q in enumerate(questions):
        user_answer = request.form.get(f"q{i}")
        correct_answer = q["answer"]
        is_correct = (user_answer == correct_answer)
        if is_correct:
            score += 1
        feedback.append({
            "question": q["question"],
            "your_answer": user_answer,
            "correct_answer": correct_answer,
            "is_correct": is_correct
        })

    return render_template_string(result_html, score=score, total=len(questions), feedback=feedback)

In [9]:
from threading import Thread

def run_app():
    app.run(port=5000)

# Start Flask in a background thread
Thread(target=run_app).start()

# Start ngrok
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("33KYZOAeQjtR0W1xKYiaFQshUjd_5aDE4cg9TB5RWg75djYJ3")
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🌍 Public URL: NgrokTunnel: "https://courtney-guardable-maurita.ngrok-free.dev" -> "http://localhost:5000"
